## Goals
* Identify logical areas for future phased expansion of the pilot program.
* Explore the calls data to look for any trends in terms of times or locations. Are there any precincts that get more than their share of calls?
* Determine if there are any trends that may be disproportionality responding to populations of high poverty, low education, housing inequities, or race/ethnicity.


In [ ]:
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import geopandas as gpd
import folium 
from folium.plugins import MarkerCluster 
from folium.plugins import FastMarkerCluster
%matplotlib inline

Import Police Call Data

In [ ]:
file = pd.ExcelFile(r"C:\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx")

In [ ]:
incidents_df = pd.read_excel(file, 'incident_types')
incidents_df.head(3)

In [ ]:
priority_df = pd.read_excel(file, 'priority')
priority_df.head(3)

In [ ]:
calls_df = pd.read_excel(file, 'call_info')
calls_df.head(3)

In [ ]:
geocodes_df = pd.read_csv(r"\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\geocodes.csv")
geocodes_df.head(3)

Look for Trends in Time/Location

In [ ]:
call_data_df = pd.merge(calls_df, geocodes_df, how = 'left', on = 'Address')
call_data_df.head(3)

In [ ]:
call_data_df.isnull().values.any()

In [ ]:
#addresses with the most incident calls, kept the ones with more than 100 calls
top_9_add = call_data_df['Address'].value_counts().nlargest(n = 9)
top_9_add

In [ ]:
top_9_df = pd.DataFrame(top_9_add).reset_index()
top_9_df = top_9_df.rename(columns = {'index' : 'Address', 'Address' : 'Count'})
top_9_df

In [ ]:
top_adds = list(top_9_df['Address'])

In [ ]:
#incident type frequency
call_data_df['Incident Type'].value_counts()

In [ ]:
#incident types at top address
add_most_calls = call_data_df.loc[call_data_df['Address'] == '639 LAFAYETTE ST']
add_most_calls

In [ ]:
add_most_calls['Incident Type'].value_counts()

In [ ]:
top_9_info = pd.merge(top_9_df, call_data_df, how = 'inner', on = 'Address')
top_9_info

In [ ]:
#count of how many rows in the calls_data_df had addresses that matched the top 9
call_data_df['Address'].isin(top_adds).value_counts()

In [ ]:
#address and incident type for the top 9
top_9_subset = top_9_info[['Address', 'Incident Type']]
top_9_subset

In [ ]:
#incident breakdown for the top 9 addresses
top_9_incidents = top_9_subset.groupby(['Incident Type']).count().reset_index()
top_9_incidents.sort_values(by = 'Address', ascending = False)

In [ ]:
top_9_subset.groupby(['Address', 'Incident Type']).count().reset_index()

In [ ]:
call_data_df

In [ ]:
precinct_month = call_data_df.groupby(['Precinct', 'Month']).count().reset_index()
precinct_month

In [ ]:
precinct_month.to_excel("output.xlsx")  

In [ ]:
top_9_info

In [ ]:
top_9_add_prec = top_9_info.groupby(['Precinct', 'Address']).count()
top_9_add_prec

In [ ]:
top_9_add_prec.to_excel("addresses.xlsx")

In [ ]:
call_data_df.groupby(['Precinct', 'Priority']).count()

In [ ]:
Precinct = pd.DataFrame(call_data_df['Precinct'].value_counts()).reset_index()
Precinct = Precinct.rename(columns = {'index':'Precinct', 'Precinct': 'Count'})
Precinct

In [ ]:
Precinct.plot.bar(x = 'Precinct', y = 'Count', rot = 0, figsize = (10, 6), color = 'Maroon', title = 'Calls per Precinct')

Connect to Census Data

In [ ]:
conn = sql.connect(r"\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\census.sqlite")
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables = (cur.fetchall())
tables

In [ ]:
census_geo = gpd.read_file(r"C:\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\tl_2019_47_tract.shp")
census_geo.head()

In [ ]:
census_geo.plot()

In [ ]:
age_sex_df = pd.read_sql("SELECT * FROM B01001;", conn)
age_sex_df

In [ ]:
age_sex_geo = age_sex_df['GEO_ID'].str.split(pat = 'US', expand = True)
age_sex_geo = age_sex_geo.rename(columns = {0 : 'drop', 1: 'GEOID'})
age_sex_geo = age_sex_geo.drop(['drop'], axis = 1)
age_sex_geo

In [ ]:
age_sex_df['GEOID'] = age_sex_geo

In [ ]:
age_sex_df.head(2)

In [ ]:
age_sex_geodf = pd.merge(age_sex_df, census_geo, how = 'inner', on = 'GEOID')
age_sex_geodf

In [ ]:
age_sex_geodf.loc[0, 'geometry']

In [ ]:
age_sex_geodf = gpd.GeoDataFrame(age_sex_geodf, 
                 crs = census_geo.crs, 
                 geometry = age_sex_geodf['geometry'])

In [ ]:
age_sex_geodf.plot(column = 'GEOID',figsize = (10,10))

In [ ]:
center = age_sex_geodf['geometry'].centroid[7888]
print(center)

In [ ]:
area_center = [center.y, center.x]
area_center

In [ ]:
map_agesex = folium.Map(location =  area_center, zoom_start = 12)
folium.GeoJson(age_sex_geodf).add_to(map_agesex)
map_agesex

In [ ]:
race_ethnicity = pd.read_sql('SELECT * FROM B03002', conn)
race_ethnicity

In [ ]:
households = pd.read_sql('SELECT * FROM B11001', conn)
households

Disconnect from Census Data

In [ ]:
cur.close()
conn.close()

In [ ]:
#to do: 
#breakdown incidents by month, time, precint, priority
#figure out for loop to find the incidents count for each address in to 9
#map the top 9 addresses